In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score


## Bli kjent med dataene

In [3]:
PATH = "/Users/Sathiesh/MachineLearning/skole_ai/fastai/fastai/courses/ml1/dataset/titanic/"

In [4]:
df_raw = pd.read_csv(f'{PATH}train.csv') 
#df_raw_test = pd.read_csv(f'{PATH}test.csv') 
#df_raw = pd.concat([df_train, df_test])

In [5]:
df_raw.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
df_raw.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [7]:
df_raw.shape #ser det at Age mangler en del. 

(891, 12)

In [8]:
df_raw.head(8)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [9]:
df_raw.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
#Hvor mange som overlevde fra de ulike klassene
survived_class = df_raw[df_raw.Survived==1]['Pclass'].value_counts()
dead_class = df_raw[df_raw.Survived==0]['Pclass'].value_counts()
df_class = pd.DataFrame([survived_class,dead_class])
df_class.index = ['Survived','Died']
display(df_class)

,1,2,3
Survived,136,87,119
Died,80,97,372


In [11]:
#Hvor mange som overlevde ut i fra kjønn 
survived_s = df_raw[df_raw.Survived == 1]['Sex'].value_counts()
dead_s = df_raw[df_raw.Survived == 0]['Sex'].value_counts()
df_sex = pd.DataFrame([survived_s, dead_s])
df_sex.index = ['Survived', 'Died']
display(df_sex)

,female,male
Survived,233,109
Died,81,468


## Jobbe med dataene(cleaning, feature selection and engineering)

In [12]:
df_raw = df_raw.drop(['PassengerId', 'Cabin', 'Ticket', 'Fare', 'Parch', 'SibSp'], axis =1) #fjerne unødvendige kolonner

In [13]:
df_raw.head()

,Survived,Pclass,Name,Sex,Age,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,S
4,0,3,"Allen, Mr. William Henry",male,35.0,S


#### Mange navn verdier i Age kolonnen

In [14]:

got= df_raw.Name.str.split(',').str[1]
df_raw['title']  = (pd.DataFrame(got).Name.str.split('\s+').str[1]) #ny kolonne med tittel 
df_raw = df_raw.drop(['Name'], axis =1) #Henter ut tittelen på alle passasjerene og sletter navnene

mean_age_by_title  =[]; 
mean_age_by_title.append(list(set(df_raw.title)))
mean_age_by_title.append(df_raw.groupby('title').Age.mean())
mean_age = np.array(mean_age_by_title).T; #Transponere matrisen

mean_age

array([['Col.', '70.0'],
       ['Mlle.', '58.0'],
       ['Jonkheer.', '40.0'],
       ['Master.', '42.0'],
       ['Rev.', '38.0'],
       ['Capt.', '48.0'],
       ['Dr.', '48.5'],
       ['Mme.', '4.57416666667'],
       ['Sir.', '21.7739726027'],
       ['Mrs.', '24.0'],
       ['the', '24.0'],
       ['Lady.', '32.3680904523'],
       ['Miss.', '35.8981481481'],
       ['Don.', '28.0'],
       ['Ms.', '43.1666666667'],
       ['Mr.', '49.0'],
       ['Major.', '33.0']],
      dtype='<U32')

In [15]:
n_rows = df_raw.shape[0] #alle radene 
n_titles = mean_age.shape[0] #alle titlene 

for i in range(0, n_rows): 
    if np.isnan(df_raw.Age[i]): 
        for j in range(0, n_titles):
            if df_raw.title[i] ==  str(mean_age[j][0]) : #skulle kanskje brukt dictonary 
                df_raw.Age[i] = mean_age[j][1].astype(np.float)

df_raw = df_raw.drop(['title'], axis=1)

/Users/Sathiesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
df_raw.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Embarked    2
dtype: int64

In [17]:
df_raw = df_raw.fillna({"Embarked": "S"}) #De fleste gikk ombord i Southampton 
print(type(df_raw.Embarked))

<class 'pandas.core.series.Series'>


In [18]:
train_cats(df_raw); #erstatter stringene med kategorier (en fast ai metode)

In [19]:
df_raw.Embarked.cat.categories

Index(['C', 'Q', 'S'], dtype='object')

In [20]:
df_trn, y_trn, nas = proc_df(df_raw, 'Survived', max_n_cat=4) #erstatte kategorier med numeriske verdier, håndtere NaN verdier og splitter den avhengige variabelen fra resten
#max_n_cats = one-hot encoding for tall under 10

In [21]:
df_trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Pclass          891 non-null int64
Age             891 non-null float64
Sex_female      891 non-null uint8
Sex_male        891 non-null uint8
Sex_nan         891 non-null uint8
Embarked_C      891 non-null uint8
Embarked_Q      891 non-null uint8
Embarked_S      891 non-null uint8
Embarked_nan    891 non-null uint8
dtypes: float64(1), int64(1), uint8(7)
memory usage: 20.1 KB


In [22]:
#Kan splitte de som er barn fra voksne
df_trn.Age = [ 0 if x > 18 else 1 for x in df_trn.Age] #bedre resultat ved å gjøre dette. 
df_trn.head(8)

,Pclass,Age,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3,0,0,1,0,0,0,1,0
1,1,0,1,0,0,1,0,0,0
2,3,0,1,0,0,0,0,1,0
3,1,0,1,0,0,0,0,1,0
4,3,0,0,1,0,0,0,1,0
5,3,0,0,1,0,0,1,0,0
6,1,0,0,1,0,0,0,1,0
7,3,1,0,1,0,0,0,1,0


In [23]:
df_trn = df_trn.drop(['Embarked_S', 'Embarked_nan'], axis=1) #Skjønner ikke helt hvorfor Embarked_nan er som en egen kolonne over

In [24]:
def split_vals(a,n): 
    return a[:n], a[n:]

In [25]:
n_valid = 200
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

X_train.shape, y_train.shape, X_valid.shape


((691, 7), (691,), (200, 7))

In [26]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
# R^2 --> (m.score()) gir ratioen til hvor god din modell er vs hvor god den naive gjennomsnitts modellen er. 


In [27]:
random_forest = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1, oob_score=True)
#For hvert tre kan man sende inn de radene som ikke blir brukt som valideringssett. 
#criterion enten gini eller entropy 

In [28]:
random_forest.fit(X_train, y_train)
print_score(random_forest)

[0.4287092328928585, 0.4242640687119285, 0.81620839363241682, 0.81999999999999995, 0.81186685962373373]


In [29]:
accuracies = cross_val_score(estimator = random_forest, X=df_trn , y=y_trn , cv = 10)
print("Random Forest:\n Accuracy:", accuracies.mean())

Random Forest:
 Accuracy: 0.8159505164


In [30]:
svm = SVC(C=2) #hard en soft margin: se tegningen. Gir bedre resultat med en gang vi øker til 2 

In [31]:
svm.fit(X_train, y_train)  
print_score(svm)

[0.433743173290677, 0.4183300132670378, 0.81186685962373373, 0.82499999999999996]


In [32]:
'''
82% would be a very good score. 84% would be an amazing score. 
If you're getting 84 then I think there's very little left that you can learn from plugging away any further on this challenge and you sh
'''

"\n82% would be a very good score. 84% would be an amazing score. \nIf you're getting 84 then I think there's very little left that you can learn from plugging away any further on this challenge and you sh\n"